In [8]:
import tensorflow as tf
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import remove_stopwords
import collections
from collections import defaultdict
import os
import re
import random
import itertools
import requests
import pickle
import heapq
import copy
from functools import reduce
import math
import pdb
from gpt2.src.score import score_model
from gpt2.src.generate_prompt import generate_prompt
from gpt2.src.encoder import get_encoder
import pickle
from py_files.Limericks import Limerick_Generate
lg = Limerick_Generate(model_name='117M')

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from gpt2.src.generate_poem_naive import generate_poem_naive
res = generate_poem_naive(model_name="117M", length=100, nsamples=10, batch_size=10, raw_text="There was a clever boy whose name is James.")

INFO:tensorflow:Restoring parameters from gpt2/models\117M\model.ckpt
======================================== SAMPLE 1 VALID 0 ========================================
======================================== SAMPLE 2 VALID 0 ========================================
======================================== SAMPLE 3 VALID 0 ========================================
======================================== SAMPLE 4 VALID 0 ========================================
======================================== SAMPLE 5 VALID 0 ========================================
======================================== SAMPLE 6 VALID 0 ========================================
======================================== SAMPLE 7 VALID 0 ========================================
======================================== SAMPLE 8 VALID 0 ========================================
======================================== SAMPLE 9 VALID 0 ========================================
=======================================

======================================== SAMPLE 82 VALID 0 ========================================
======================================== SAMPLE 83 VALID 0 ========================================
======================================== SAMPLE 84 VALID 0 ========================================
======================================== SAMPLE 85 VALID 0 ========================================
======================================== SAMPLE 86 VALID 0 ========================================
======================================== SAMPLE 87 VALID 0 ========================================
======================================== SAMPLE 88 VALID 0 ========================================
======================================== SAMPLE 89 VALID 0 ========================================
======================================== SAMPLE 90 VALID 0 ========================================
======================================== SAMPLE 91 VALID 0 ========================================


======================================== SAMPLE 163 VALID 0 ========================================
======================================== SAMPLE 164 VALID 0 ========================================
======================================== SAMPLE 165 VALID 0 ========================================
======================================== SAMPLE 166 VALID 0 ========================================
======================================== SAMPLE 167 VALID 0 ========================================
======================================== SAMPLE 168 VALID 0 ========================================
======================================== SAMPLE 169 VALID 0 ========================================
======================================== SAMPLE 170 VALID 0 ========================================
======================================== SAMPLE 171 VALID 0 ========================================
======================================== SAMPLE 172 VALID 0 ===============================

======================================== SAMPLE 244 VALID 0 ========================================
======================================== SAMPLE 245 VALID 0 ========================================
======================================== SAMPLE 246 VALID 0 ========================================
======================================== SAMPLE 247 VALID 0 ========================================
======================================== SAMPLE 248 VALID 0 ========================================
======================================== SAMPLE 249 VALID 0 ========================================
======================================== SAMPLE 250 VALID 0 ========================================
======================================== SAMPLE 251 VALID 0 ========================================
======================================== SAMPLE 252 VALID 0 ========================================
======================================== SAMPLE 253 VALID 0 ===============================

======================================== SAMPLE 325 VALID 0 ========================================
======================================== SAMPLE 326 VALID 0 ========================================
======================================== SAMPLE 327 VALID 0 ========================================
======================================== SAMPLE 328 VALID 0 ========================================
======================================== SAMPLE 329 VALID 0 ========================================
======================================== SAMPLE 330 VALID 0 ========================================
======================================== SAMPLE 331 VALID 0 ========================================
======================================== SAMPLE 332 VALID 0 ========================================
======================================== SAMPLE 333 VALID 0 ========================================
======================================== SAMPLE 334 VALID 0 ===============================

...

======================================== SAMPLE 5203 VALID 9 ========================================
======================================== SAMPLE 5204 VALID 9 ========================================
======================================== SAMPLE 5205 VALID 9 ========================================
======================================== SAMPLE 5206 VALID 9 ========================================
======================================== SAMPLE 5207 VALID 9 ========================================
======================================== SAMPLE 5208 VALID 9 ========================================
======================================== SAMPLE 5209 VALID 9 ========================================
======================================== SAMPLE 5210 VALID 9 ========================================
======================================== SAMPLE 5211 VALID 9 ========================================
======================================== SAMPLE 5212 VALID 9 =====================

======================================== SAMPLE 5283 VALID 9 ========================================
======================================== SAMPLE 5284 VALID 9 ========================================
======================================== SAMPLE 5285 VALID 9 ========================================
======================================== SAMPLE 5286 VALID 9 ========================================
======================================== SAMPLE 5287 VALID 9 ========================================
======================================== SAMPLE 5288 VALID 9 ========================================
======================================== SAMPLE 5289 VALID 9 ========================================
======================================== SAMPLE 5290 VALID 9 ========================================
======================================== SAMPLE 5291 VALID 9 ========================================
======================================== SAMPLE 5292 VALID 9 =====================

======================================== SAMPLE 5363 VALID 9 ========================================
======================================== SAMPLE 5364 VALID 9 ========================================
======================================== SAMPLE 5365 VALID 9 ========================================
======================================== SAMPLE 5366 VALID 9 ========================================
======================================== SAMPLE 5367 VALID 9 ========================================
======================================== SAMPLE 5368 VALID 9 ========================================
======================================== SAMPLE 5369 VALID 9 ========================================
======================================== SAMPLE 5370 VALID 9 ========================================
======================================== SAMPLE 5371 VALID 9 ========================================
======================================== SAMPLE 5372 VALID 9 =====================

======================================== SAMPLE 5443 VALID 9 ========================================
======================================== SAMPLE 5444 VALID 9 ========================================
======================================== SAMPLE 5445 VALID 9 ========================================
======================================== SAMPLE 5446 VALID 9 ========================================
======================================== SAMPLE 5447 VALID 9 ========================================
======================================== SAMPLE 5448 VALID 9 ========================================
======================================== SAMPLE 5449 VALID 9 ========================================
======================================== SAMPLE 5450 VALID 9 ========================================
======================================== SAMPLE 5451 VALID 9 ========================================
======================================== SAMPLE 5452 VALID 9 =====================

======================================== SAMPLE 5523 VALID 9 ========================================
======================================== SAMPLE 5524 VALID 9 ========================================
======================================== SAMPLE 5525 VALID 9 ========================================
======================================== SAMPLE 5526 VALID 9 ========================================
======================================== SAMPLE 5527 VALID 9 ========================================
======================================== SAMPLE 5528 VALID 9 ========================================
======================================== SAMPLE 5529 VALID 9 ========================================
======================================== SAMPLE 5530 VALID 9 ========================================
======================================== SAMPLE 5531 VALID 9 ========================================
======================================== SAMPLE 5532 VALID 9 =====================

In [17]:
print(res)

None


In [ ]:
def save_poem_to_text_file(poems, file_name):
    f = open(file_name,'a') 
    for poem in poems:
        for line in poem:
            line = re.sub(r'[^\w\s]','',line)
            f.write(line)
            f.write("\n")
        f.write("\n")
save_poem_to_text_file(res, 'py_files/saved_objects/silly_poems_2.txt')

In [23]:
# poem = pickle.load('gpt2.pkl')
f = open('gpt2.pkl', 'rb')
poems = pickle.load(f)

In [24]:
print(poems)

[262, 37934, 994, 29742, 866, 438, 198, 3152, 15133, 4692, 290, 288, 260, 560, 26, 198, 1870, 1793, 9859, 1474, 284, 607, 438, 64, 2415, 11, 198, 1890, 1793, 290, 607, 1672, 11386, 13, 8117, 1752, 373, 281, 21848, 2415, 3706, 37392]


In [44]:
f = open('py_files/saved_objects/silly_poems_with_templates.txt', 'rb')
for line in f:
    line = str(line).split(',')
    for i in range(len(line)):
        line[i] = re.sub(r"[^A-Za-z]+", '', line[i])
    line[0] = line[0][1:]
    print(' '.join(line))

there once was a woman enjoying her life in lamarrn
who had men in a carrn
i took up one tunern
of a new moonrn
and was in a th barrn
rn
there was an enthusiastic man that lived in dakarrn
who saw things from another starrn
i found out one afternoonrn
that the dark moonrn
and lost in the dark scarrn
rn
there once was an enthusiastic woman named romainern
who told stories about the painrn
she felt off one kindrn
of a high windrn
and fell off a high planern
rn
there once was an enthusiastic man named casern
who did wonders for the birthplacern
i got a reportrn
from a local courtrn
and asked for a special gracern
rn
there once was an enthusiastic woman named germainern
who laid eggs on a grainrn
i came up one mindrn
in a un kindrn
and lay in a u veinrn
rn
there once was an enthusiastic woman named germainern
who wrote books in every domainrn
she experienced off one mankindrn
in any great whirlwindrn
and fell in a great rainrn
rn
there was an enthusiastic man that lived in morainern
who co